In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from tqdm.notebook import tqdm

In [2]:
web_logs = pd.read_csv('2022-04-01T12_df_web_logs.csv')

In [3]:
web_logs.head(3)

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7


In [4]:
web_logs_week = web_logs[
    (web_logs['date'] >= '2022-03-01')
    & (web_logs['date'] < '2022-03-08')
]
web_logs_week.shape

(245851, 4)

In [5]:
alpha = 0.05
sample_size = 1000

mean = web_logs_week['load_time'].mean()
std = web_logs_week['load_time'].std()

eff = 0.01
users = web_logs_week['user_id'].unique()

In [44]:
outliers = [0.0002, 0.002, 0.02, 0.1, 0.2]
result = []

for quant in tqdm(outliers):
    pvalues = []

    for _ in range(1000):
        users_a, users_b = np.random.choice(users, (2, sample_size), replace=False)
        a = web_logs_week.query('user_id in @users_a').load_time
        b = web_logs_week.query('user_id in @users_b').load_time * (1 + eff)

        a_mask = (a > np.quantile(a, quant / 2)) & (a < np.quantile(a, 1 - quant / 2))
        b_mask = (b > np.quantile(b, quant / 2)) & (b < np.quantile(b, 1 - quant / 2))
        group_a = a[a_mask]
        group_b = b[b_mask]

        pvalue = stats.ttest_ind(group_a, group_b).pvalue
        pvalues.append(pvalue)

    power = np.mean((np.array(pvalues) < alpha).astype(int))
    result.append([quant, power])

    print(f'Для значения квантиля в {quant} мощность теста равна {power}')
    
result_sorted = sorted(result, key=lambda power: power[1])
answer_list = [i + 1 for i, v in enumerate(result_sorted)]
print(f'''
ответ: {"".join(map(str, reversed(answer_list)))}.
''')

  0%|          | 0/5 [00:00<?, ?it/s]

Для значения квантиля в 0.0002 мощность теста равна 0.096
Для значения квантиля в 0.002 мощность теста равна 0.359
Для значения квантиля в 0.02 мощность теста равна 0.94
Для значения квантиля в 0.1 мощность теста равна 0.952
Для значения квантиля в 0.2 мощность теста равна 0.967

ответ: 54321.

